<a href="https://colab.research.google.com/github/koseki3104/biginner_practice202506/blob/main/%E5%8C%BB%E7%99%82%E8%B2%BB%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/SIGNATE/【2025.06】医療保険の費用帯予測')


In [ ]:
import pandas as pd
test = pd.read_csv("test.csv")
print(test.info())
train = pd.read_csv('train.csv')
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        400 non-null    int64  
 1   age       400 non-null    int64  
 2   sex       400 non-null    object 
 3   bmi       400 non-null    float64
 4   children  400 non-null    int64  
 5   smoker    400 non-null    object 
 6   region    400 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 22.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        1600 non-null   int64  
 1   age       1600 non-null   int64  
 2   sex       1600 non-null   object 
 3   bmi       1600 non-null   float64
 4   children  1600 non-null   int64  
 5   smoker    1600 non-null   object 
 6   region    1600 non-null   object 
 7   charges   1600 